Import required libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import requests

Get the URL text and read it into a Soup Object (Beautiful Soup)

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text
soup = BS(source, 'lxml')    

Find the Table from the Wikipedia page which has the Postal Codes, Borough and Neighbourhood information; Append the rows into a list

In [3]:
table = soup.find('table', class_ = 'wikitable sortable')
row_data = []

for row in table.find_all("tr"):
    row_data.append(row.text)

#print(parsed_table_data) 

Split (delimiter = '\n') each row list value so that each cell value is an individual element of a list.
Since there is a new line character at the start and end of each row, clean up each list element by deleting the empty charaters

In [4]:
cell_data = []

for i in row_data:
    words = i.split('\n')
    del words[0]
    del words[3]
    cell_data.append(words)
    
del cell_data[0]

Import the data from the list into a DataFrame

In [5]:
from pandas import DataFrame

df = DataFrame.from_records(cell_data, columns=['Postcode', 'Borough', 'Neighborhood'])

Clean up the Borough and Neighbourhood values where they are 'Not assigned'

In [6]:
# Drop records where Borough is 'Not assigned'
df_clean = df.drop(df[(df.Borough == 'Not assigned')].index)

#Where Neighbourhood is 'Not assigned', set to the Borough value
df_clean.loc[df_clean['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df_clean.Borough

#Reset Index
df_clean = df_clean.reset_index(drop=True)

Group the Neighbourhoods with the same PostCode, separated by comma

In [7]:
df_final = df_clean.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
#df_final

Get the shape of the dataframe

In [8]:
df_final.shape

(103, 3)

Install GeoCoder

In [9]:
#!conda install -c conda-forge geocoder --yes

Since Geocoder isnt giving results, import the data from theGeospatial dataset provided in the course

In [9]:
!wget -q -O '/Users/PreethiKrishnan/Desktop/projects/Coursera_Capstone/Toronto_Geospatial_data.csv' http://cocl.us/Geospatial_data
#print('Data downloaded!')

/bin/sh: wget: command not found


Load data into Dataframe

In [10]:
toronto_data = pd.read_csv("/Users/PreethiKrishnan/Desktop/projects/Coursera_Capstone/Toronto_Geospatial_data.csv")
toronto_data.head()

FileNotFoundError: [Errno 2] File b'Toronto_Geospatial_data.csv' does not exist: b'Toronto_Geospatial_data.csv'

In [ ]:
toronto_data.shape

Concatenate Latitude and Longitude with original Toronto DF

In [ ]:
toronto_data.columns=['PostalCode', 'Latitude', 'Longitude']

toronto_lat_long = pd.concat([df_final, toronto_data], axis=1)
del toronto_lat_long['PostalCode']
toronto_lat_long.head()


In [ ]:
toronto_lat_long.loc[toronto_lat_long['Neighborhood'] == 'Woodbine Heights',  'Latitude']

In [ ]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Get the latitude and longitude of Toronto

In [ ]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Create a Map of Toronto

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
map_toronto

Superimpose the Toronto neighbourhoods onto the map

In [ ]:
for lat, lng, borough, neighborhood in zip(toronto_lat_long['Latitude'], toronto_lat_long['Longitude'], toronto_lat_long['Borough'], toronto_lat_long['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials

In [ ]:
CLIENT_ID = 'NTM1GC2C0GRDRTHGP3ZFAGSHRKWMTAJNJXY0I4G4G2XJDMKM' # your Foursquare ID
CLIENT_SECRET = 'MVHFSY3A2OQHVLVUNQOCDGPYHRPL0Y3WQAZZG3CFIDFKUAEP' # your Foursquare Secret
VERSION = '20190523' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Get the lat and lng values of the Clairlea, Golden Mile, Oakridge neighbourhoods in the dataframe

In [ ]:
neighborhood_latitude = toronto_lat_long.loc[7, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_lat_long.loc[7, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_lat_long.loc[7, 'Neighborhood'] # neighborhood name

print('Neighbourhood : {} ; Lat: {} ; Lng: {}'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))
#print('Latitude and longitude values of {} neighbourhood(s) are {}, {}.'.format(neighborhood_name, 
#                                                               neighborhood_latitude, 
#                                                               neighborhood_longitude))

Get the top 10 venues in the Clairlea, Golden Mile, Oakridge neighbourhoods, within a radius of 500 meters

In [ ]:
limit = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

url

In [ ]:
results = requests.get(url).json()
results

Function to extract the category of the venue

In [ ]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Structure the venue details into a dataframe

In [ ]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

Explore all neighbourhoods

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_lat_long['Neighborhood'],
                                   latitudes=toronto_lat_long['Latitude'],
                                   longitudes=toronto_lat_long['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

What are the types of Venues found in Toronto

In [ ]:
toronto_venues['Venue Category'].unique()

Number of venues in each Neighbourhood

In [ ]:
toronto_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()

In [ ]:
toronto_venues.loc[toronto_venues['Neighborhood'] == 'Adelaide, King, Richmond',  'Venue Category'].unique()

Analyze each neighbourhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot

In [ ]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.columns[:]


In [ ]:
# Move neighbors column to the first

neigh = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neigh)
toronto_onehot.head()

Print each neighborhood along with the top 5 most common venues

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Sort venues and display top 10 venues as columns

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Cluster Neighbourhoods

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Combine cluster and the top 10 venues for each neighbourhood

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:
toronto_merged = toronto_lat_long

# merge toronto_grouped with toronto_data to add latitude/longitude for each Neighbourhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Drop rows where Cluster Labels is NaN and convert the cluster label to int

In [ ]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(-1)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(str)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].replace('-1', 'Invalid')
#toronto_merged[pd.isnull(toronto_merged['Cluster Labels'])]

In [ ]:
toronto_cluster_clean = toronto_merged.drop(toronto_merged[(toronto_merged['Cluster Labels'] == 'Invalid')].index)

In [ ]:
#toronto_cluster_clean['Cluster Labels'].astype(int)
toronto_cluster_clean = toronto_cluster_clean.astype({'Cluster Labels': int})
toronto_cluster_clean['Cluster Labels'].unique()
type(toronto_cluster_clean['Cluster Labels'][0])

Visualize the clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_cluster_clean['Latitude'], toronto_cluster_clean['Longitude'], toronto_cluster_clean['Neighborhood'], toronto_cluster_clean['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters